In [4]:
import pymongo

from pymongo import MongoClient

client = MongoClient()

client = MongoClient('localhost', 27017)

In [5]:
client.list_database_names()

['NBA', 'admin', 'config', 'local']

In [6]:
db = client['NBA']

In [7]:
import pandas as pd

In [8]:
#how to read from mongodb to pandas

In [9]:
from nba_api.stats.endpoints import playbyplayv2
from nba_api.stats.endpoints import leaguegamelog

In [11]:
seasons = ['2014-15','2015-16','2016-17','2017-18','2018-19', '2019-20', '2020-21', '2021-22']

In [12]:
lgls = []

for season in seasons:
    lgls.append(leaguegamelog.LeagueGameLog(season=season).get_data_frames()[0])

In [13]:
lgl =pd.concat(lgls[1:])

In [15]:
lgl

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22015,1610612744,GSW,Golden State Warriors,0021500003,2015-10-27,GSW vs. NOP,W,240,41,...,35,56,29,8,7,20,29,111,16,1
1,22015,1610612740,NOP,New Orleans Pelicans,0021500003,2015-10-27,NOP @ GSW,L,240,35,...,25,33,21,9,3,19,26,95,-16,1
2,22015,1610612739,CLE,Cleveland Cavaliers,0021500002,2015-10-27,CLE @ CHI,L,240,38,...,39,50,26,5,7,11,21,95,-2,1
3,22015,1610612741,CHI,Chicago Bulls,0021500002,2015-10-27,CHI vs. CLE,W,240,37,...,40,47,13,6,10,13,22,97,2,1
4,22015,1610612765,DET,Detroit Pistons,0021500001,2015-10-27,DET @ ATL,W,240,37,...,36,59,23,5,3,15,15,106,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,22021,1610612765,DET,Detroit Pistons,0022100579,2022-01-06,DET @ MEM,L,240,34,...,29,39,23,7,3,13,14,88,-30,1
1136,22021,1610612740,NOP,New Orleans Pelicans,0022100580,2022-01-06,NOP vs. GSW,W,240,35,...,38,46,20,7,5,18,18,101,5,1
1137,22021,1610612744,GSW,Golden State Warriors,0022100580,2022-01-06,GSW @ NOP,L,240,37,...,34,42,22,13,5,14,24,96,-5,1
1138,22021,1610612756,PHX,Phoenix Suns,0022100581,2022-01-06,PHX vs. LAC,W,240,44,...,42,63,22,5,7,8,20,106,17,1


In [16]:
game_log = db.game_log

In [17]:
game_log.insert_many(lgl.to_dict('records'))

In [ ]:
game_log.find()

In [23]:
g_ids = NBA['historical_pbp'].distinct('GAME_ID')

In [46]:
len(g_ids)

7059

In [24]:
updated_g_ids = lgl['GAME_ID'].astype(int).unique()

In [25]:
g_ids_to_update = set(updated_g_ids).difference(set(g_ids))

In [26]:
len(g_ids_to_update)

566

In [33]:
g_ids_to_update = [f'00{gid}' for gid in g_ids_to_update]

In [35]:
from tqdm import tqdm
from time import sleep
pbp = []
for game_id in tqdm(g_ids_to_update):
    pbp.append(playbyplayv2.PlayByPlayV2(game_id))
    sleep(1)

100%|██████████████████████████████████████████████████████████████████| 566/566 [15:41<00:00,  1.66s/it]


In [37]:
pbp_dfs = [pbp[i].get_data_frames()[0] for i in range(len(pbp))]

In [38]:
pbp_dfs_full = pd.concat(pbp_dfs)

In [42]:
pbp_dfs_full['GAME_ID'] = pbp_dfs_full['GAME_ID'].astype(int)

In [43]:
pbp_dfs_full

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,22100001,2,12,0,1,7:37 PM,12:00,None,Start of 1st Period (7:37 PM EST),None,...,None,None,0,0,None,NaN,None,None,None,0
1,22100001,4,10,0,1,7:37 PM,12:00,Jump Ball Lopez vs. Claxton: Tip to Antetokounmpo,None,None,...,Nets,BKN,4,203507,Giannis Antetokounmpo,1.610613e+09,Milwaukee,Bucks,MIL,1
2,22100001,7,2,1,1,7:38 PM,11:42,MISS Allen 27' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1
3,22100001,8,4,0,1,7:38 PM,11:39,None,None,Durant REBOUND (Off:0 Def:1),...,None,None,0,0,None,NaN,None,None,None,1
4,22100001,9,6,2,1,7:38 PM,11:27,Antetokounmpo S.FOUL (P1.T1) (N.Sago),None,None,...,Nets,BKN,1,0,None,NaN,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,22100577,660,5,1,4,12:26 AM,0:05,Barnes Bad Pass Turnover (P2.T11),None,Gallinari STEAL (1 STL),...,Hawks,ATL,0,0,None,NaN,None,None,None,1
478,22100577,662,6,28,4,12:26 AM,0:05,Hield Personal Take Foul (P4.PN) (J.Goble),None,None,...,Hawks,ATL,1,0,None,NaN,None,None,None,1
479,22100577,664,3,11,4,12:27 AM,0:05,None,None,Gallinari Free Throw 1 of 2 (15 PTS),...,None,None,0,0,None,NaN,None,None,None,1
480,22100577,665,3,12,4,12:27 AM,0:05,None,None,Gallinari Free Throw 2 of 2 (16 PTS),...,None,None,0,0,None,NaN,None,None,None,1


In [45]:
NBA['historical_pbp'].insert_many(pbp_dfs_full.to_dict('records'))

In [47]:
g_ids_new = NBA['historical_pbp'].distinct('GAME_ID')

In [48]:
len(g_ids_new)

7625